## Loading training & test datasets

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('image_emotions4.csv')
df

,filename,angry,sad,happy,fear,surprise,disgust,neutral,emotion
0,image_06637.jpg,1,0,0,0,0,0,0,angry
1,image_06638.jpg,1,0,0,0,0,0,0,angry
2,image_06639.jpg,0,0,0,0,0,1,0,disgust
3,image_06641.jpg,0,0,0,0,0,0,1,neutral
4,image_06642.jpg,0,1,0,0,0,0,0,sad
...,...,...,...,...,...,...,...,...,...
5831,image_08036.jpg,1,0,0,0,0,0,0,angry
5832,image_08035.jpg,0,0,0,0,0,0,1,neutral
5833,image_08038.jpg,0,1,0,0,0,0,0,sad
5834,image_08028.jpg,0,0,0,1,0,0,0,fear


In [3]:
df_angry = df[['filename', 'angry']]
df_angry['label'] = df_angry['angry'] + 1
df_angry

<ipython-input-3-d58687d7adbb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_angry['label'] = df_angry['angry'] + 1


,filename,angry,label
0,image_06637.jpg,1,2
1,image_06638.jpg,1,2
2,image_06639.jpg,0,1
3,image_06641.jpg,0,1
4,image_06642.jpg,0,1
...,...,...,...
5831,image_08036.jpg,1,2
5832,image_08035.jpg,0,1
5833,image_08038.jpg,0,1
5834,image_08028.jpg,0,1


In [6]:
df_test = pd.read_csv('cropped_images_annotated.csv')
df_test

,filename,angry,sad,happy,neutral,label
0,cropped_images/img_1.jpg,0,0,0,1,Neutral
1,cropped_images/img_2.jpg,0,0,0,1,Neutral
2,cropped_images/img_3.jpg,0,0,0,1,Neutral
3,cropped_images/img_4.jpg,0,0,0,1,Neutral
4,cropped_images/img_5.jpg,1,0,0,0,Angry
...,...,...,...,...,...,...
761,cropped_images/img_762.jpg,0,0,0,0,Sad
762,cropped_images/img_763.jpg,0,0,0,1,Neutral
763,cropped_images/img_764.jpg,0,0,0,1,Neutral
764,cropped_images/img_765.jpg,0,0,1,0,Happy


In [7]:
df_T_angry = df_test[['filename', 'angry']]
df_T_angry['label'] = df_T_angry['angry'] + 1
df_T_angry

<ipython-input-7-06c8aa8fcb6b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_T_angry['label'] = df_T_angry['angry'] + 1


,filename,angry,label
0,cropped_images/img_1.jpg,0,1
1,cropped_images/img_2.jpg,0,1
2,cropped_images/img_3.jpg,0,1
3,cropped_images/img_4.jpg,0,1
4,cropped_images/img_5.jpg,1,2
...,...,...,...
761,cropped_images/img_762.jpg,0,1
762,cropped_images/img_763.jpg,0,1
763,cropped_images/img_764.jpg,0,1
764,cropped_images/img_765.jpg,0,1


## Setup

### Import packages

In [4]:
import cv2
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torchvision
from PIL import Image
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models

from torch.utils.data import Dataset, DataLoader
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
from torchvision.io import read_image
from torchvision.transforms import ToTensor, ToPILImage, Normalize, Compose

### Set device

In [5]:
device = None

if device == None:
    if torch.cuda.is_available():
        print("CUDA used")
        device = torch.device('cuda')

        # Should be > 0
        print(torch.cuda.device_count())

        # Index of device used (can be 0)
        print(torch.cuda.current_device())

        # GPU location
        print(torch.cuda.device(0))

        # Name of GPU
        print(torch.cuda.get_device_name(0))

    else:
        print("CPU used")
        device = torch.device('cpu')

# Force CPU use
# device = torch.device('cpu')

CPU used


### Load data

In [9]:
training_data = []

for i in range(0, len(df_angry)):
    image, label = df_angry.iloc[i]['filename'], df_angry.iloc[i]['label']
    path = 'portrait_faces/' + image
    new_row = [path, int(label)]
    training_data.append(new_row)

In [10]:
print(len(training_data))

5836


In [11]:
train_set = training_data[0:int(len(training_data) * 0.8)]
validation_set = training_data[int(len(training_data) * 0.8):]
print(len(train_set) + len(validation_set))

5836


In [12]:
test_set_filename = []
test_set = []

for i in range(0, len(df_T_angry)):
    image, label = df_T_angry.iloc[i]['filename'], df_T_angry.iloc[i]['label']
    new_row = [image, int(label)]
    test_set_filename.append(image)
    test_set.append(new_row)

In [13]:
print(len(test_set))

766


In [14]:
# Create a map for the labels and their codes
labels = ['not_angry', 'angry']
class_map = {}
i = 1
    
for label in labels:
    class_map[i] = label
    i += 1

In [15]:
print(class_map)

{1: 'not_angry', 2: 'angry'}


In [17]:
# Class for loading of the data
class own_data(Dataset):
    def __init__(self, train=0, transform=None):
        if train == 0:
            self.data = train_set
        elif train == 1:
            self.data = validation_set
        else:
            self.data = test_set
            
        self.transform = transform
        self.to_pil = ToPILImage()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, class_id = self.data[idx]
        image = read_image(img_path)
        
        if self.transform:
            image = self.transform(self.to_pil(image))
        
        return image, class_id

## Model

### Initializing Resnet50 pretrained model

In [18]:
weights = models.ResNet50_Weights.IMAGENET1K_V2
preprocess = weights.transforms()
resnet50 = models.resnet50(weights=weights)
print(resnet50)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [19]:
# Freeze model parameters
for param in resnet50.parameters():
    param.requires_grad = False

for param in resnet50.layer4.parameters():
    param.requires_grad = True
    
# Change the final layer of ResNet50 Model for Transfer Learning
fc_inputs = resnet50.fc.in_features
resnet50.fc = nn.Sequential(
    nn.Linear(fc_inputs, 256),
    nn.Dropout(0.2, inplace=False),
    nn.BatchNorm1d(num_features=256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
    nn.Linear(256, 7),
    nn.LogSoftmax(dim=1) # For using NLLLoss()/kldiv
)

print(resnet50)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [20]:
# Define the transform function
# test_transform = transforms.Compose([
#     transforms.Resize(232), 
#     transforms.ToTensor(), 
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
# ])

# train_transform = transforms.Compose([
#     transforms.Resize(256), 
#     transforms.CenterCrop(size=224), 
#     transforms.ToTensor(), 
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
# ])

In [21]:
# Initialize the batch size
batch_size = 64

In [22]:
# Load the train and test data
train_data = own_data(0, preprocess)
validation_data = own_data(1, preprocess)
test_data = own_data(2, preprocess)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

### Setting model for 'angry' to Resnet50

In [23]:
model_angry = resnet50
model_angry.train()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [24]:
## Some loss functions
nll = nn.NLLLoss()

cross = nn.CrossEntropyLoss()

kldiv = nn.KLDivLoss(reduction="batchmean")

In [26]:
# Some optimizers
sgd = optim.SGD(model_angry.parameters(), lr=0.001, momentum=0.9)

adam = optim.Adam(model_angry.parameters(), lr=3e-4)

RMSProp = optim.RMSprop(model_angry.parameters(), lr=1e-3, momentum=0.9)

adamw = optim.AdamW(model_angry.parameters(), lr=3e-4)

### Define training & test functions for model

In [27]:
# Define the train model function
def train_model(train_loader, model, loss_function, optimizer, batch_size, device=None):
    for batch, (image, label) in enumerate(train_loader):
        # Move labels and images to GPU
        image = image.to(device)
        label = label.to(device)

        optimizer.zero_grad()
        output = model(image)
        label = label - 1
        loss_func = loss_function(output, label)
        loss_func.backward()
        optimizer.step()
        
        if batch % 64 == 0:
            print('[%5d] loss: %.3f' % (batch + 1, loss_func.item()))

In [28]:
# Test function
def validate_model(validation_loader, model, loss_function, batch_size, device=None):
    size = len(validation_loader.dataset)
    validation_loss, correct = 0, 0
    
    with torch.no_grad():
        for image, label in validation_loader:
            # Move labels and images to GPU
            image = image.to(device)
            label = label.to(device)
            
            output = model(image)
            label = label - 1
            validation_loss += loss_function(output, label).item()
            correct += (output.argmax(1) == label).type(torch.float).sum().item()
    
    validation_loss /= batch_size
    correct /= size
    print(f"Test error: \n Accuracy: {(100 * correct):>0.01f}%, Avg loss: {validation_loss:>8f} \n")

### Run model for epochs

In [30]:
epochs = 7
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    model_angry.train()
    train_model(train_loader, model_angry, nll, adam, batch_size, device=device)
    model_angry.eval()
    validate_model(validation_loader, model_angry, nll, batch_size, device=device)
print("Done!")

Epoch 1
-------------------------------
[    1] loss: 1.881
[   65] loss: 1.660
Test error: 
 Accuracy: 90.3%, Avg loss: 0.352072 

Epoch 2
-------------------------------
[    1] loss: 0.911
[   65] loss: 0.366
Test error: 
 Accuracy: 88.4%, Avg loss: 0.334791 

Epoch 3
-------------------------------
[    1] loss: 0.212
[   65] loss: 0.188
Test error: 
 Accuracy: 91.1%, Avg loss: 0.137767 

Epoch 4
-------------------------------
[    1] loss: 0.135
[   65] loss: 0.062
Test error: 
 Accuracy: 87.8%, Avg loss: 0.168032 

Epoch 5
-------------------------------
[    1] loss: 0.113
[   65] loss: 0.083
Test error: 
 Accuracy: 88.4%, Avg loss: 0.302901 

Epoch 6
-------------------------------
[    1] loss: 0.107
[   65] loss: 0.027
Test error: 
 Accuracy: 89.2%, Avg loss: 0.205964 

Epoch 7
-------------------------------
[    1] loss: 0.029
[   65] loss: 0.053
Test error: 
 Accuracy: 90.2%, Avg loss: 0.148281 

Done!


## Test model

In [31]:
model_angry.eval()

def test_model(test_loader, model, device=None):
    predicted_labels = []
    
    with torch.no_grad():
        for image, label in test_loader:
            # Move to GPU
            image = image.to(device)
            label = label.to(device)
            
            output = model(image)
            predicted_label = (output.argmax(1)).cpu().detach().numpy()
            predicted_labels.extend(predicted_label)
    
#     predictions = [x - 1 for x in predicted_labels]
    return predicted_labels

In [32]:
predictions = test_model(test_loader, model_angry, device)

In [34]:
rijks_prediction = pd.DataFrame(
    {'filename': test_set_filename,
     'label': predictions
    })

rijks_prediction

,filename,label
0,cropped_images/img_1.jpg,0
1,cropped_images/img_2.jpg,0
2,cropped_images/img_3.jpg,0
3,cropped_images/img_4.jpg,0
4,cropped_images/img_5.jpg,0
...,...,...
761,cropped_images/img_762.jpg,0
762,cropped_images/img_763.jpg,0
763,cropped_images/img_764.jpg,0
764,cropped_images/img_765.jpg,0


In [36]:
rijks_prediction.rename(columns = {'label':'predicted_label'}, inplace=True)
rijks_prediction['actual_label'] = df_T_angry['angry']
rijks_prediction

,filename,predicted_label,actual_label
0,cropped_images/img_1.jpg,0,0
1,cropped_images/img_2.jpg,0,0
2,cropped_images/img_3.jpg,0,0
3,cropped_images/img_4.jpg,0,0
4,cropped_images/img_5.jpg,0,1
...,...,...,...
761,cropped_images/img_762.jpg,0,0
762,cropped_images/img_763.jpg,0,0
763,cropped_images/img_764.jpg,0,0
764,cropped_images/img_765.jpg,0,0


In [40]:
correct = (rijks_prediction['predicted_label'] == 1)
accuracy_angry = (correct.sum() / (rijks_prediction['actual_label'] == 1).size) * 100
accuracy_angry

0.13054830287206268

In [39]:
rijks_prediction.to_csv('predictions/resnet50_predictions_angry.csv', index=False)

## Preprocessing 'sad' dataset

In [48]:
df = pd.read_csv('image_emotions4.csv')
df

,filename,angry,sad,happy,fear,surprise,disgust,neutral,emotion
0,image_06637.jpg,1,0,0,0,0,0,0,angry
1,image_06638.jpg,1,0,0,0,0,0,0,angry
2,image_06639.jpg,0,0,0,0,0,1,0,disgust
3,image_06641.jpg,0,0,0,0,0,0,1,neutral
4,image_06642.jpg,0,1,0,0,0,0,0,sad
...,...,...,...,...,...,...,...,...,...
5831,image_08036.jpg,1,0,0,0,0,0,0,angry
5832,image_08035.jpg,0,0,0,0,0,0,1,neutral
5833,image_08038.jpg,0,1,0,0,0,0,0,sad
5834,image_08028.jpg,0,0,0,1,0,0,0,fear


In [49]:
df = df[['filename', 'sad']]
df['label'] = df['sad'] + 1
df

<ipython-input-49-54f45bdf8f5a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['sad'] + 1


,filename,sad,label
0,image_06637.jpg,0,1
1,image_06638.jpg,0,1
2,image_06639.jpg,0,1
3,image_06641.jpg,0,1
4,image_06642.jpg,1,2
...,...,...,...
5831,image_08036.jpg,0,1
5832,image_08035.jpg,0,1
5833,image_08038.jpg,1,2
5834,image_08028.jpg,0,1


### Load data

In [50]:
data = []

for i in range(0, len(df)):
    image, label = df.iloc[i]['filename'], df.iloc[i]['label']
    path = 'portrait_faces/' + image
    new_row = [path, int(label)]
    data.append(new_row)

In [51]:
print(len(data))

5836


In [52]:
train_set = data[0:int(len(data) * 0.8)]
test_set = data[int(len(data) * 0.8):]
print(len(train_set) + len(test_set))

5836


In [53]:
# Create a map for the labels and their codes
labels = ['not_sad', 'sad']
class_map = {}
i = 1
    
for label in labels:
    class_map[i] = label
    i += 1

In [54]:
print(class_map)

{1: 'not_sad', 2: 'sad'}


In [55]:
# Load the train and test data
train_data = own_data(0, train_transform)
test_data = own_data(1, test_transform)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

### Setting model for 'sad' to Resnet50

In [56]:
model_sad = resnet50
model_sad.train()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

### Run model for epochs

In [57]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    model_sad.train()
    train_model(train_loader, model_sad, nll, adam, batch_size, device=device)
    model_sad.eval()
    test_model(test_loader, model_sad, nll, batch_size, device=device)
print("Done!")

Epoch 1
-------------------------------
[    1] loss: 6.587
[   65] loss: 1.018
Test error: 
 Accuracy: 75.8%, Avg loss: 0.289804 

Epoch 2
-------------------------------
[    1] loss: 0.828
[   65] loss: 0.722
Test error: 
 Accuracy: 77.4%, Avg loss: 0.247217 

Epoch 3
-------------------------------
[    1] loss: 0.342
[   65] loss: 0.273
Test error: 
 Accuracy: 76.9%, Avg loss: 0.235425 

Epoch 4
-------------------------------
[    1] loss: 0.152
[   65] loss: 0.248
Test error: 
 Accuracy: 77.1%, Avg loss: 0.406988 

Epoch 5
-------------------------------
[    1] loss: 0.043
[   65] loss: 0.010
Test error: 
 Accuracy: 78.6%, Avg loss: 0.264505 

Done!


## Preprocessing 'happy' dataset

In [58]:
df = pd.read_csv('image_emotions4.csv')
df

,filename,angry,sad,happy,fear,surprise,disgust,neutral,emotion
0,image_06637.jpg,1,0,0,0,0,0,0,angry
1,image_06638.jpg,1,0,0,0,0,0,0,angry
2,image_06639.jpg,0,0,0,0,0,1,0,disgust
3,image_06641.jpg,0,0,0,0,0,0,1,neutral
4,image_06642.jpg,0,1,0,0,0,0,0,sad
...,...,...,...,...,...,...,...,...,...
5831,image_08036.jpg,1,0,0,0,0,0,0,angry
5832,image_08035.jpg,0,0,0,0,0,0,1,neutral
5833,image_08038.jpg,0,1,0,0,0,0,0,sad
5834,image_08028.jpg,0,0,0,1,0,0,0,fear


In [59]:
df = df[['filename', 'happy']]
df['label'] = df['happy'] + 1
df

<ipython-input-59-028b67a0a642>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['happy'] + 1


,filename,happy,label
0,image_06637.jpg,0,1
1,image_06638.jpg,0,1
2,image_06639.jpg,0,1
3,image_06641.jpg,0,1
4,image_06642.jpg,0,1
...,...,...,...
5831,image_08036.jpg,0,1
5832,image_08035.jpg,0,1
5833,image_08038.jpg,0,1
5834,image_08028.jpg,0,1


### Load data

In [60]:
data = []

for i in range(0, len(df)):
    image, label = df.iloc[i]['filename'], df.iloc[i]['label']
    path = 'portrait_faces/' + image
    new_row = [path, int(label)]
    data.append(new_row)

In [61]:
print(len(data))

5836


In [62]:
train_set = data[0:int(len(data) * 0.8)]
test_set = data[int(len(data) * 0.8):]
print(len(train_set) + len(test_set))

5836


In [63]:
# Create a map for the labels and their codes
labels = ['not_happy', 'happy']
class_map = {}
i = 1
    
for label in labels:
    class_map[i] = label
    i += 1

In [64]:
print(class_map)

{1: 'not_happy', 2: 'happy'}


In [65]:
# Load the train and test data
train_data = own_data(0, train_transform)
test_data = own_data(1, test_transform)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

### Setting model for 'happy'

In [66]:
model_happy = resnet50
model_happy.train()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [67]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    model_happy.train()
    train_model(train_loader, model_happy, nll, adam, batch_size, device=device)
    model_happy.eval()
    test_model(test_loader, model_happy, nll, batch_size, device=device)
print("Done!")

Epoch 1
-------------------------------
[    1] loss: 4.148
[   65] loss: 0.278
Test error: 
 Accuracy: 95.1%, Avg loss: 0.069642 

Epoch 2
-------------------------------
[    1] loss: 0.191
[   65] loss: 0.147
Test error: 
 Accuracy: 95.1%, Avg loss: 0.062225 

Epoch 3
-------------------------------
[    1] loss: 0.261
[   65] loss: 0.111
Test error: 
 Accuracy: 95.1%, Avg loss: 0.058075 

Epoch 4
-------------------------------
[    1] loss: 0.130
[   65] loss: 0.211
Test error: 
 Accuracy: 95.1%, Avg loss: 0.052214 

Epoch 5
-------------------------------
[    1] loss: 0.091
[   65] loss: 0.109
Test error: 
 Accuracy: 95.1%, Avg loss: 0.054064 

Done!
